In [ ]:
import os
import requests
from PIL import Image
from io import BytesIO

# Set up SerpAPI key (ensure to use environment variable for security in production)
SERPAPI_KEY = "################"

def search_and_download_images(query, num_images, out_dir='images'):
    """
    Searches for copyright-free images using SerpAPI and downloads them as PNGs.

    Args:
        query (str): The search query.
        num_images (int): Number of images to download.
        out_dir (str): The root output directory for images.
    """
    # Usage rights: "public domain" or "Creative Commons"
    params = {
        "q": query,
        "tbm": "isch",
        "tbs": "sur:fm",  # Searches for freely usable images
        "api_key": SERPAPI_KEY,
    }
    url = "https://serpapi.com/search.json"
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Failed to retrieve data from SerpAPI. Status code: {response.status_code}")
        return

    data = response.json()
    image_results = data.get('images_results', [])
    if not image_results:
        print("No images found for query:", query)
        return

    # Prepare category directory
    category_dir = os.path.join(out_dir, query.replace(" ", "_"))
    os.makedirs(category_dir, exist_ok=True)

    downloaded = 0
    for i, result in enumerate(image_results):
        if downloaded >= num_images:
            break
        image_url = result.get('original')
        if not image_url:
            continue
        try:
            image_response = requests.get(image_url, timeout=10)
            if image_response.status_code == 200:
                with Image.open(BytesIO(image_response.content)) as img:
                    img = img.convert('RGBA')
                    image_path = os.path.join(category_dir, f'{query.replace(" ", "_")}_{downloaded+1}.png')
                    img.save(image_path, format='PNG')
                    print(f'Downloaded and converted {image_path}')
                    downloaded += 1
            else:
                print(f'Failed to download {image_url}')
        except Exception as e:
            print(f'Error processing {image_url}: {e}')

if __name__ == "__main__":
    # Example usage
    categories = ["Demographic Induvisuals"]
    for category in categories:
        search_and_download_images(category, num_images=200)